### Урок 4. Рекомендательные системы на основе контента

### -- Автор: Шенк Евгений Станиславович

# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

## Задание 1. Напишите свою функцию prefilter_items

In [1]:
import pandas as pd

data = pd.read_csv('../data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [2]:
# Сохранено в utils.ru
'''
def prefilter_items(data, take_n_popular=5000):
    """Предфильтрация товаров"""

    # 0. Удаляем позиции у которых количество = 0
    idx = data.loc[data['quantity'] == 0].index
    data.drop(idx, inplace = True)
    data.reset_index(drop=True, inplace=True)
    data['usd_per_unit'] = data['sales_value'] / data['quantity']

    # 1.2. Удаление товаров, со средней ценой < 1$ and > 30$
    df_grouped = data.groupby(by=['item_id']).mean()
    idx_less_1 = data.loc[data['item_id'].isin(df_grouped.loc[df_grouped['usd_per_unit'] < 1].index)].index
    idx_more_30 = data.loc[data['item_id'].isin(df_grouped.loc[df_grouped['usd_per_unit'] > 30].index)].index
    data = data.loc[~data.index.isin(np.concatenate([idx_less_1, idx_more_30]))]
    data.reset_index(drop=True, inplace=True)

    # 3. Придумайте свой фильтр (убраны товары, которые не продавались 1 год)
    df_grouped = data.groupby(by=['item_id']).max().sort_values('day')
    idx_sold_12m = data.loc[data['item_id'].isin(df_grouped.loc[df_grouped['day'] > (df_grouped['day'].max() - 366)].index)].index
    data = data.loc[data.index.isin(idx_sold_12m)]
    data.reset_index(drop=True, inplace=True)

    # 4. Выбор топ-N самых популярных товаров (N = take_n_popular)
    df_grouped = data.groupby(by=['item_id']).sum().sort_values('quantity', ascending = False)
    idx_top_k = data.loc[data['item_id'].isin(df_grouped.head(take_n_popular).index)].index
    data = data.loc[data.index.isin(idx_top_k)]
    data.reset_index(drop=True, inplace=True)

    return data
'''

'\ndef prefilter_items(data, take_n_popular=5000):\n    """Предфильтрация товаров"""\n\n    # 0. Удаляем позиции у которых количество = 0\n    idx = data.loc[data[\'quantity\'] == 0].index\n    data.drop(idx, inplace = True)\n    data.reset_index(drop=True, inplace=True)\n    data[\'usd_per_unit\'] = data[\'sales_value\'] / data[\'quantity\']\n\n    # 1.2. Удаление товаров, со средней ценой < 1$ and > 30$\n    df_grouped = data.groupby(by=[\'item_id\']).mean()\n    idx_less_1 = data.loc[data[\'item_id\'].isin(df_grouped.loc[df_grouped[\'usd_per_unit\'] < 1].index)].index\n    idx_more_30 = data.loc[data[\'item_id\'].isin(df_grouped.loc[df_grouped[\'usd_per_unit\'] > 30].index)].index\n    data = data.loc[~data.index.isin(np.concatenate([idx_less_1, idx_more_30]))]\n    data.reset_index(drop=True, inplace=True)\n\n    # 3. Придумайте свой фильтр (убраны товары, которые не продавались 1 год)\n    df_grouped = data.groupby(by=[\'item_id\']).max().sort_values(\'day\')\n    idx_sold_12m = d

В корне проекта создайте модуль *src*. Положите функцию *prefilter_items* в файл *src/utils.py*  

Далее запустите код ниже и перейдите к заданию 2 (в конце ноутбука)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [4]:
data = pd.read_csv('../data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Warm start
train_items = data_train['item_id'].unique()
train_users = data_train['user_id'].unique()

data_test = data_test[data_test['item_id'].isin(train_items)]
data_test = data_test[data_test['user_id'].isin(train_users)]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
item_features = pd.read_csv('../data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [7]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

C:\Users\Eugene\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
D:\Auress\Data_Science\Python_for_DS\Recommendation_systems\src\utils.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['usd_per_unit'] = data['sales_value'] / data['quantity']


Decreased # items from 86865 to 5000


In [8]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,818981,819063,819112,819255,819304,819308,819330,819518,819594,819765,...,15717057,15741861,15742123,15831255,15926712,15926775,15926844,15926886,15926927,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [10]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [11]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 1.72 s


-----

# Домашнее задание

1. Перенесите метрики из ДЗ 1 src/metrics.py
3. Создайте модуль src/recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src/recommenders.py
4. Проверьте, что все модули корректно импортируются

Выполнение: сохранены  
    src/utils.py  
    src/metrics.py  
    src/recommenders.py  
И так же загружены на GitHub вместе с этим файлом    

In [12]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

In [13]:
# Сохранено в src/recommenders.py 
'''
class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    # data = pd.read_csv('../data/retail_train.csv')
    # item_features = pd.read_csv('../data/product.csv')

    def __init__(self, data, item_features, weighting=True):

        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != 999999]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != 999999]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()

        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)

        # Словарь {item_id: 0/1}. 0/1 - факт принадлежности товара к СТМ
        self.item_id_to_ctm = dict(zip(item_features['item_id'], item_features['brand'].isin(['Private']).astype(int)))

        # Own recommender обучается до взвешивания матрицы
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)

    @staticmethod
    def prepare_matrix(data, val='quantity'):

        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values=val,  # Можно пробовать другие варианты
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

        return user_item_matrix

    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)

        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model

    def get_similar_items_recommendation(self, user, filter_ctm=True, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        rec_model = self.model.similar_items(self.itemid_to_id[user], N=N)
        recs = [x[0] for x in rec_model][1:]  # получаем список рекомендаций
        rec_to_itemid = [self.id_to_itemid[x] for x in recs]  # переводим в изначальные id

        if filter_ctm:
            ctm_list = [self.item_id_to_ctm[x] for x in rec_to_itemid]  # Список является или нет товар СТМ

            try:
                idx = ctm_list.index(1)  # Берем первый товар СТМ
            except ValueError:
                idx = 0  # либо просто первый, если СТМ в списке не оказалось
        else:
            idx = 0

        res = rec_to_itemid[idx]

        return res

    def get_similar_users_recommendation(self, user, N_users=5, N_items_per_user=5, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        rec_model_users = self.model.similar_users(self.userid_to_id[user], N=N_users+1)
        recs_users = [x[0] for x in rec_model_users][1:]  # получаем список Юзеров

        total_recs = []
        for similar_user in recs_users:
            own = ItemItemRecommender(K=1, num_threads=4)  # K - кол-во билжайших соседей
            own.fit(csr_matrix(self.user_item_matrix).T.tocsr())

            recs = own.recommend(userid=self.userid_to_id[similar_user],  # Находим купленые ими товары
                                     user_items=csr_matrix(self.user_item_matrix).tocsr(),  # на вход user-item matrix
                                     N=N_items_per_user,
                                     filter_already_liked_items=False,
                                     filter_items=None,
                                     recalculate_user=False)
            total_recs.append(recs)

        total_recs = [item for sublist in total_recs for item in sublist]  # делаем общий список товаров
        total_recs = sorted(total_recs, key=lambda l: l[1], reverse=True)  # выбираем товары с большим мкором

        res = [self.id_to_itemid[x[0]] for x in total_recs][:N]  # берем нужное кол-во и переводим в изначальные id

        return res
    '''

'\nclass MainRecommender:\n    """Рекоммендации, которые можно получить из ALS\n\n    Input\n    -----\n    user_item_matrix: pd.DataFrame\n        Матрица взаимодействий user-item\n    """\n\n    # data = pd.read_csv(\'../data/retail_train.csv\')\n    # item_features = pd.read_csv(\'../data/product.csv\')\n\n    def __init__(self, data, item_features, weighting=True):\n\n        # Топ покупок каждого юзера\n        self.top_purchases = data.groupby([\'user_id\', \'item_id\'])[\'quantity\'].count().reset_index()\n        self.top_purchases.sort_values(\'quantity\', ascending=False, inplace=True)\n        self.top_purchases = self.top_purchases[self.top_purchases[\'item_id\'] != 999999]\n\n        # Топ покупок по всему датасету\n        self.overall_top_purchases = data.groupby(\'item_id\')[\'quantity\'].count().reset_index()\n        self.overall_top_purchases.sort_values(\'quantity\', ascending=False, inplace=True)\n        self.overall_top_purchases = self.overall_top_purchases[self

----

Проверка, что все работает

In [14]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [15]:
ctm = item_features[item_features['brand'] == 'Private'].item_id.unique() # СТМ = товары под брендом Private

popularity = data_train[~data_train['item_id'].isin(ctm)].groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
popularity.sort_values('quantity', ascending=False, inplace=True)

popularity = popularity[popularity['item_id'] != 999999]

popularity = popularity.groupby('user_id').head(5)

popularity.sort_values('user_id', ascending=False, inplace=True)
popularity.head(2)

,user_id,item_id,quantity
406028,2500,859237,16
406112,2500,974108,5


In [16]:
mr = MainRecommender(data_train, item_features, weighting=True)

In [17]:
popularity['similar_recommendation'] = popularity['item_id'].apply(lambda x: MainRecommender.get_similar_items_recommendation(self=mr, user=x, filter_ctm=False, N=5))
popularity.head(5)

,user_id,item_id,quantity,similar_recommendation
406028,2500,859237,16,991951
406112,2500,974108,5,15596279
406168,2500,1063739,8,1010051
406170,2500,1065538,10,1034176
406232,2500,6534030,6,6533608


In [18]:
popularity['similar_recommendation'] = popularity['item_id'].apply(lambda x: MainRecommender.get_similar_items_recommendation(self=mr, user=x, filter_ctm=True, N=5))
popularity.head(5)

,user_id,item_id,quantity,similar_recommendation
406028,2500,859237,16,991951
406112,2500,974108,5,995242
406168,2500,1063739,8,1135044
406170,2500,1065538,10,1034176
406232,2500,6534030,6,6533608


In [19]:
user = 2500
MainRecommender.get_similar_users_recommendation(self=mr, user=user, N_users=2, N_items_per_user=5, N=5)

[835578, 1030093, 1115069, 6979579, 820486]